In [1]:
import joblib
import pandas as pd
import numpy as np
import spacy
import os
from src.classes.qadataset import QADataset
from datasets import load_dataset, Dataset
from collections import defaultdict
import random
from tqdm.auto import tqdm
from typing import List, Dict, Tuple, Union
import re
from copy import deepcopy

In [2]:
d = load_dataset("Seongill/Trivia_missing_5_small", split="train")

In [4]:
trivia_dataset = pd.DataFrame(load_dataset("Seongill/Trivia_missing_5", split="train"))
#nq_dataset= pd.DataFrame(load_dataset("Seongill/nq", split="test"))

In [6]:
from datasets import load_dataset

dataset = pd.DataFrame(load_dataset("Seongill/Trivia_missing_10", split="train"))

subset_questions = trivia_dataset.question.tolist()
sub_dataset = dataset[dataset["question"].isin(subset_questions)]
print(len(sub_dataset))

q_to_new_ctxs = dict()
for i, row in tqdm(sub_dataset.iterrows()):
    q_to_new_ctxs[row["question"]] = row["ctxs"]

11313


0it [00:00, ?it/s]

In [7]:
train, test = QADataset.load("TriviaTrain"), QADataset.load("TriviaTest")
res = dict()
grouped_examples = defaultdict(list)
cnt = 0
for ex in train.examples:
    grouped_examples[ex.get_example_answer_type()].append(ex)
for group, ex_list in grouped_examples.items():
    print(f"Answer Type: {group} | Size of Group: {len(ex_list)}")

Read 87622 examples from /data/seongil/datasets/normalized/TriviaTrain.jsonl.gz
Read 11313 examples from /data/seongil/datasets/normalized/TriviaTest.jsonl.gz
Answer Type: PERSON | Size of Group: 22369
Answer Type: LOCATION | Size of Group: 10777
Answer Type: None | Size of Group: 42978
Answer Type: ORGANIZATION | Size of Group: 8237
Answer Type: NUMERIC | Size of Group: 2070
Answer Type: DATE | Size of Group: 1191


In [8]:
import spacy
nlp = spacy.load("en_core_web_lg")
spacy.prefer_gpu()

True

In [9]:
unique_mentions = []
mention_to_ner = dict()
ent_to_answer_unique = defaultdict(list)

for k,v in grouped_examples.items():
    if k == None:
        print("Length of None:", len(v))
        continue
    else:
        entities = []
        for ex in v:    
            answers = [d.text for d in ex.gold_answers]
            entities.extend(answers)
        entities = list(set(entities))
        ent_to_answer_unique[k] = entities
for group, ex_list in grouped_examples.items():
    print(f"Answer Type: {group} | Size of Group: {len(ex_list)}")
print("-"*50)
for group, ex_list in ent_to_answer_unique.items():
    print(f"Answer Type: {group} | Size of Group: {len(ex_list)}")

for k,v in grouped_examples.items():
    if k == None:
        print("Length of None:", len(v))
        continue
    else:
        entities = []
        for ex in v:    
            answers = [d.text for d in ex.gold_answers]
            entities.extend(answers)
        entities = list(set(entities))
        unique_mentions.extend(entities)
        for ent in entities:
            mention_to_ner[ent] = k
unique_mentions = list(set(unique_mentions))
print("Total Mentions:", len(unique_mentions))
print(len(mention_to_ner))

Length of None: 42978
Answer Type: PERSON | Size of Group: 22369
Answer Type: LOCATION | Size of Group: 10777
Answer Type: None | Size of Group: 42978
Answer Type: ORGANIZATION | Size of Group: 8237
Answer Type: NUMERIC | Size of Group: 2070
Answer Type: DATE | Size of Group: 1191
--------------------------------------------------
Answer Type: PERSON | Size of Group: 78587
Answer Type: LOCATION | Size of Group: 26150
Answer Type: ORGANIZATION | Size of Group: 41415
Answer Type: NUMERIC | Size of Group: 1270
Answer Type: DATE | Size of Group: 2354
Length of None: 42978
Total Mentions: 149724
149724


In [8]:
ent_type_vector_map = defaultdict(list)
embeds = []
for i in tqdm(range(0, len(unique_mentions), 4000)):
    batch = unique_mentions[i:i+4000]
    for mention, _doc in zip(batch, list(nlp.pipe(batch))):
        embeds.append(_doc.vector)
        ent_type_vector_map[mention_to_ner[mention]].append(_doc.vector)
len(embeds)

  0%|          | 0/38 [00:00<?, ?it/s]

149724

In [9]:
entity_vector_map = dict()
for mention, vec in zip(unique_mentions, embeds):
    entity_vector_map[mention] = vec

subset_questions = trivia_dataset["question"].tolist()
print("before:", len(test.examples))
test.examples = list(filter(lambda x: x.query in subset_questions, test.examples))
print("after:", len(test.examples))

before: 11313
after: 3771


In [10]:
ans = []
for ex in test.examples:
    ans.extend([d.text for d in ex.gold_answers])
ans = list(set(ans))
for i in tqdm(range(0, len(ans), 4000)):
    batch = ans[i:i+4000]
    for mention, _doc in zip(batch, list(nlp.pipe(batch))):
        entity_vector_map[mention] = _doc.vector

  0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
import random
import numpy as np
from numpy.linalg import norm
def check_cosine_similarity(arr1, arr2, thres):
    cosine_similarity = np.dot(arr1, arr2) / (norm(arr1) * norm(arr2))
    return (cosine_similarity <= thres) and (cosine_similarity >= 0.6)
    
def find_non_identical_random_entity(ent_type, origin: Union[str, List[str]], thres: float=0.8):
    subsets = ent_to_answer_unique[ent_type]
    #random_entities = random.sample(subsets, 100 if len(subsets)>100 else len(subsets))
    for entity in subsets:#random_entities:
        origin = origin if not isinstance(origin, list) else random.choice(origin)
        origin_vec, entity_vec = entity_vector_map[origin], entity_vector_map[entity]
        if check_cosine_similarity(origin_vec, entity_vec, thres):
            return entity
    return random.choice(subsets)

def find_random_entity(ent_type, origin: Union[str, List[str]]):
    subsets = ent_to_answer_unique[ent_type]
    random_entity = random.choice(subsets)
    if isinstance(origin, list):
        while random_entity in origin:
            random_entity = random.choice(subsets)
    else:
        while random_entity == origin:
            random_entity = random.choice(subsets)
    return random_entity

In [12]:
question_map = dict()
random_subs, similar_subs, questions, gold_answers, ent_types = [], [], [], [], []
for ex in tqdm(test.examples):
    query, answers, ent_type = ex.query, [d.text for d in ex.gold_answers], ex.get_example_answer_type()
    questions.append(query)
    gold_answers.append(answers)
    if ent_type:
        random_sub = find_random_entity(ent_type, answers)
        similar_sub = find_non_identical_random_entity(ent_type, answers)
    else:
        random_sub, similar_sub = None, None
    random_subs.append(random_sub)
    similar_subs.append(similar_sub)
    ent_types.append(ent_type)

  0%|          | 0/3771 [00:00<?, ?it/s]

/tmp/ipykernel_3274570/2366755019.py:5: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity = np.dot(arr1, arr2) / (norm(arr1) * norm(arr2))


In [31]:
df = pd.DataFrame({"question": questions, "gold_answers": gold_answers, "random_sub": random_subs, "similar_sub": similar_subs, "ent_type": ent_types})
trivia_dataset = trivia_dataset.merge(df, on="question", how="left")
trivia_dataset.drop(columns=["gold_answers"], inplace=True)

In [45]:
res = []
ans_count, in_context_count = [], []
for i, row in tqdm(trivia_dataset.iterrows(), total=len(trivia_dataset)):
    answers_in_context = []
    ans_count.append(len(row["answers"]))
    for answer in row["answers"]:
        for ctx in row["ctxs"]:
            if answer in ctx["text"]:
                answers_in_context.append(answer)
    in_context_count.append(len(answers_in_context))
    res.append(answers_in_context)
print("Average Answers in Context:", np.mean(in_context_count))
print("Average Answers:", np.mean(ans_count))
trivia_dataset["answers_in_context"] = res

  0%|          | 0/3771 [00:00<?, ?it/s]

Average Answers in Context: 2.1349774595597983
Average Answers: 13.550251922566959


In [ ]:
for ex in tqdm(test.examples):
    answer_type = ex.get_example_answer_type()
    if not answer_type:
        res[ex.query] = None
        continue
    item = random.choice(grouped_examples[answer_type])
    while item == ex:
        item = random.choice(grouped_examples[answer_type])
    res[ex.query] = item.gold_answers[0].text
    cnt += 1
trivia_dataset["substitute"] = trivia_dataset["question"].apply(lambda x: res[x])

In [ ]:
res = []
for row in tqdm(df.iterrows()):
    res.append(find_non_identical_random_entity(mention_to_ner[row[1]["answer"]], row[1]["answer"]))

In [54]:
def find_answer_in_context(answer_text: str, context: str):
    if isinstance(context, str):
        context_spans = [
            (m.start(), m.end())
            for m in re.finditer(re.escape(answer_text.lower()), context.lower())
        ]
        return context_spans
    else:
        return [""]
def update_context_with_substitution_string(
    context: str, originals:List[str], substitution: str, replace_every_string=True
) -> str:
    replace_spans = []
    for orig_answer in originals:
        replace_spans.extend(find_answer_in_context(orig_answer, context))
    replace_strs = set([context[span[0] : span[1]] for span in replace_spans])
    for replace_str in replace_strs:
        context = context.replace(replace_str, substitution)
    return context

Number Conflict: 1426
Average of hasanswer: 3.3044285335454786
Averaged Number of Replaced Contexts: 2.3043478260869565


In [55]:
trivia_dataset.head(1)

,question,answers,ctxs,has_answer,random_sub,similar_sub,ent_type,answers_in_context,num_has_answer_ctx,answer_candidates,valid_answer_candidates
0,What type of dance shoe has a specially harden...,"[Tap Dance, Tapdance, Soft shoe dance, Tap dan...","[{'hasanswer': False, 'id': '371852', 'score':...",True,None,None,None,[],1,0,0


In [64]:
is_conflict, num_replace, random_new_ctxs, num_ans = [], [], [], []
TOPK = 5
half = False
buffer = False
for row_ in trivia_dataset.iterrows():
    row = row_[1]
    ctxs = deepcopy(row["ctxs"][:TOPK])
    if row["random_sub"] is None:
        is_conflict.append(False)
        num_replace.append(0)
        random_new_ctxs.append(ctxs)
    else:
        num_answers = row["valid_answer_candidates"]
        if num_answers > 1:
            if buffer and half:
                is_conflict.append(False)
                num_replace.append(0)
                random_new_ctxs.append(ctxs)
                buffer = False
                continue
            buffer = True
            random.seed(42)
            substitute = row["random_sub"]
            ctxs_answer_in = [idx for idx, d in enumerate(ctxs) if d["nli_has_answer"]]
            replace_num = random.randint(1, num_answers-1)
            sampled_idx = random.sample(ctxs_answer_in, replace_num)
            for idx, ctx in enumerate(ctxs):
                if idx in sampled_idx:
                    ctxs[idx]["text"] = update_context_with_substitution_string(ctxs[idx]["text"], row["answers"], substitute)
            random_new_ctxs.append(ctxs)
            is_conflict.append(True)
            num_replace.append(replace_num)
        else:
            is_conflict.append(False)
            num_replace.append(0)
            random_new_ctxs.append(ctxs)
print(f"Number Conflict: {sum(is_conflict)}")
print(f"Average of hasanswer: {np.mean([sum([int(d['hasanswer']) for d in ctxs]) for ctxs in random_new_ctxs])}")
print(f"Averaged Number of Replaced Contexts: {np.mean([n for n in num_replace if n > 0])}")

Number Conflict: 699
Average of hasanswer: 1.9345001325908247
Averaged Number of Replaced Contexts: 1.251788268955651


In [59]:
is_conflict, num_replace, similar_new_ctxs, num_ans = [], [], [], []
TOPK = 5
half = False
buffer = False
for row_ in trivia_dataset.iterrows():
    row = row_[1]
    ctxs = deepcopy(row["ctxs"][:TOPK])
    if row["random_sub"] is None:
        is_conflict.append(False)
        num_replace.append(0)
        similar_new_ctxs.append(ctxs)
    else:
        num_answers = row["valid_answer_candidates"]
        if num_answers > 1:
            if buffer and half:
                is_conflict.append(False)
                num_replace.append(0)
                similar_new_ctxs.append(ctxs)
                buffer = False
                continue
            buffer = True
            random.seed(42)
            substitute = row["random_sub"]
            ctxs_answer_in = [idx for idx, d in enumerate(ctxs) if d["nli_has_answer"] and d["hasanswer"]]
            replace_num = random.randint(1, num_answers-1)
            sampled_idx = random.sample(ctxs_answer_in, replace_num)
            for idx, ctx in enumerate(ctxs):
                if idx in sampled_idx:
                    ctxs[idx]["text"] = update_context_with_substitution_string(ctxs[idx]["text"], row["answers"], substitute)
            similar_new_ctxs.append(ctxs)
            is_conflict.append(True)
            num_replace.append(replace_num)
        else:
            is_conflict.append(False)
            num_replace.append(0)
            similar_new_ctxs.append(ctxs)
print(f"Number Conflict: {sum(is_conflict)}")
print(f"Average of hasanswer: {np.mean([sum([int(d['hasanswer']) for d in ctxs]) for ctxs in similar_new_ctxs])}")
print(f"Averaged Number of Replaced Contexts: {np.mean([n for n in num_replace if n > 0])}")

Number Conflict: 699
Average of hasanswer: 1.9345001325908247
Averaged Number of Replaced Contexts: 1.251788268955651


In [73]:
new_df = trivia_dataset.copy()
new_df = new_df.drop(columns=["ctxs","has_answer"], axis=1)
new_df["random_substituted_ctxs"] = random_new_ctxs
new_df["similar_substituted_ctxs"] = similar_new_ctxs
new_df["is_conflict"] = is_conflict
new_df["num_replace"] = num_replace
name = f"TriviaQA_conflict_{TOPK}_{'full' if not half else 'half'}_small"

In [68]:
new_df = new_df.drop(columns=["answers_in_context","num_has_answer_ctx", "answer_candidates","valid_answer_candidates"], axis=1)

In [71]:
new_df.to_csv("temp.csv", index=False)

In [ ]:
new_df[(new_df.is_conflict) & (new_df.num_answer > 2)]

In [77]:
sample_df = new_df.sample(1000, random_state=42)
sample_df.is_conflict.sum()

187

In [80]:
sample_df.drop(["random_sub", "similar_sub", "ent_type",
                "answers_in_context", "num_has_answer_ctx",
                "answer_candidates", "valid_answer_candidates",
                "random_substituted_ctxs"], axis=1, inplace=True)
sample_df["ctxs"] = sample_df["similar_substituted_ctxs"]
sample_df.drop("similar_substituted_ctxs", axis=1, inplace=True)

In [81]:
Dataset.from_pandas(sample_df).push_to_hub("TriviaQA_conflict_5_small_v2")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
Dataset.from_pandas(new_df).push_to_hub(name)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

In [100]:
3610-1729

1881

In [46]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = "cross-encoder/qnli-electra-base"
model = AutoModelForSequenceClassification.from_pretrained(nli_model).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(nli_model)

In [51]:
import torch
model.eval()
res = []
for i, row in tqdm(trivia_dataset.iterrows(), total=len(trivia_dataset)):
    query = row["question"]
    ctxs = [ctx["text"] for ctx in row["ctxs"]]
    features = tokenizer([query]*len(ctxs), ctxs, padding="max_length", truncation=True, max_length=256, return_tensors="pt").to("cuda")
    with torch.no_grad():
        scores = torch.nn.functional.sigmoid(model(**features).logits).detach().cpu().numpy()
        idxs = np.where(scores > 0.5)[0]
        for j, ctx in enumerate(row["ctxs"]):
            if j in idxs:
                ctx["nli_has_answer"] = True
            else:
                ctx["nli_has_answer"] = False
        res.append(idxs.size)

  0%|          | 0/3771 [00:00<?, ?it/s]

In [53]:
trivia_dataset["num_has_answer_ctx"] = trivia_dataset["ctxs"].apply(lambda x: sum([d["hasanswer"] for d in x]))
trivia_dataset["answer_candidates"] = res
trivia_dataset["valid_answer_candidates"] = trivia_dataset["ctxs"].apply(lambda x: sum([True if d["hasanswer"] and d["nli_has_answer"] else False for d in x]))
trivia_dataset[(trivia_dataset.valid_answer_candidates > 2)]

,question,answers,ctxs,has_answer,random_sub,similar_sub,ent_type,answers_in_context,num_has_answer_ctx,answer_candidates,valid_answer_candidates
7,In the Blandings Castle stories by P G Wodehou...,"[Chazer, Piggeh, Sus (genus), Pig, Pigs, 🐷, 🐖, 🐽]","[{'hasanswer': True, 'id': '4601349', 'score':...",True,None,None,None,"[Pig, Pig, Pig, Pigs, Pigs]",5,4,4
11,"The voice of Bugs Bunny, Daffy Duck, Porky Pig...","[The Man of a Thousand Voices, Sy, the Little ...","[{'hasanswer': True, 'id': '549983', 'score': ...",True,PAUL STEPHENSON,Mel C.,PERSON,"[Melvin Jerome Blanc, Melvin Jerome Blanc, Mel...",5,4,4
18,Ecuador has a border with Peru and which other...,"[República de Colombia, Etymology of Colombia,...","[{'hasanswer': True, 'id': '119939', 'score': ...",True,Gelibolu Peninsula,"Garden Creek, New Brunswick",LOCATION,"[Colombia, Colombia, Colombia, Colombia]",4,3,3
19,What was the first national park in the USA,"[Yellowstone National Park Archives, Yellowsto...","[{'hasanswer': True, 'id': '289806', 'score': ...",True,Punjab (Province),Kentucky River,LOCATION,"[Yellowstone, Yellowstone, Yellowstone, Yellow...",5,4,4
25,Who designed the Queen's wedding dress?,"[Norman Bishop Hartnell, Norman Hartnell, Sir ...","[{'hasanswer': True, 'id': '15572631', 'score'...",True,The Pied Piper of R&B,John McEnroe,PERSON,"[Norman Hartnell, Norman Hartnell, Norman Hart...",3,4,3
...,...,...,...,...,...,...,...,...,...,...,...
3751,What was the Troggs most famous hit?,"[Wild Thing (song), Wild Thing, Wild Thing (di...","[{'hasanswer': True, 'id': '1615929', 'score':...",True,None,None,None,"[Wild Thing, Wild Thing, Wild Thing, Wild Thing]",4,4,4
3752,Cholecystectomy is the surgical removal of whi...,"[Galblader, Gal blader, Corpus vesicae biliari...","[{'hasanswer': True, 'id': '2944151', 'score':...",True,None,None,None,[Cholecyst],5,5,5
3756,What animal is the Scandinavian Christmas Julb...,"[Nanny goat, Dairy goat, Kid (goat), Nanny Goa...","[{'hasanswer': True, 'id': '6670299', 'score':...",True,Lady Chatterley's Lover (book),Kung Zhu,PERSON,"[Goat, Goat, Goat, Goats]",4,3,3
3761,"Also known as honey wine, what is the name of ...","[Hydromel, Metheglins, Pyment, Rhodomel, Honey...","[{'hasanswer': True, 'id': '454926', 'score': ...",True,None,None,None,[Mead],5,4,4


In [58]:
trivia_dataset[(trivia_dataset.valid_answer_candidates > 1) & (trivia_dataset.ent_type)]

,question,answers,ctxs,has_answer,random_sub,similar_sub,ent_type,answers_in_context,num_has_answer_ctx,answer_candidates,valid_answer_candidates
11,"The voice of Bugs Bunny, Daffy Duck, Porky Pig...","[The Man of a Thousand Voices, Sy, the Little ...","[{'hasanswer': True, 'id': '549983', 'score': ...",True,PAUL STEPHENSON,Mel C.,PERSON,"[Melvin Jerome Blanc, Melvin Jerome Blanc, Mel...",5,4,4
18,Ecuador has a border with Peru and which other...,"[República de Colombia, Etymology of Colombia,...","[{'hasanswer': True, 'id': '119939', 'score': ...",True,Gelibolu Peninsula,"Garden Creek, New Brunswick",LOCATION,"[Colombia, Colombia, Colombia, Colombia]",4,3,3
19,What was the first national park in the USA,"[Yellowstone National Park Archives, Yellowsto...","[{'hasanswer': True, 'id': '289806', 'score': ...",True,Punjab (Province),Kentucky River,LOCATION,"[Yellowstone, Yellowstone, Yellowstone, Yellow...",5,4,4
22,Which 19th century king sired ten illegitimate...,"[William IV (disambiguation), Wilhelm IV, Will...","[{'hasanswer': True, 'id': '1296740', 'score':...",True,Francis Albert Augustus Charles Emmanuel,John McEnroe,PERSON,"[William IV, William IV, William IV]",3,2,2
25,Who designed the Queen's wedding dress?,"[Norman Bishop Hartnell, Norman Hartnell, Sir ...","[{'hasanswer': True, 'id': '15572631', 'score'...",True,The Pied Piper of R&B,John McEnroe,PERSON,"[Norman Hartnell, Norman Hartnell, Norman Hart...",3,4,3
...,...,...,...,...,...,...,...,...,...,...,...
3732,In which modern country is the ancient city of...,"[Jordan (country), Al-Urdunn, أردنّ, JOrdan, U...","[{'hasanswer': True, 'id': '7118350', 'score':...",True,C. J. Parker,Gazi Mustafa Kemal Pasha,PERSON,"[Jordan, Jordan, Jordan, Jordan]",4,2,2
3739,"Opened in 1637, in which city was the first pu...","[Sestiere (Venice), Venice, Venedig, Districts...","[{'hasanswer': True, 'id': '9805483', 'score':...",True,WYE,Italian capital,LOCATION,"[Venice, Venice, Venice, Venice]",4,2,2
3743,Which newsreader famously sat on a Lesbian whe...,"[Nicholas Newton Henshall Witchell, Nick Witch...","[{'hasanswer': False, 'id': '6336484', 'score'...",True,(Norman) Foster,Arnold Daniel Palmer,PERSON,"[Nicholas Witchell, Nicholas Witchell]",2,4,2
3756,What animal is the Scandinavian Christmas Julb...,"[Nanny goat, Dairy goat, Kid (goat), Nanny Goa...","[{'hasanswer': True, 'id': '6670299', 'score':...",True,Lady Chatterley's Lover (book),Kung Zhu,PERSON,"[Goat, Goat, Goat, Goats]",4,3,3


In [1]:
from datasets import load_dataset
import pandas as pd
dataset = pd.DataFrame(load_dataset("Seongill/squad_adversarial_thres1"))


In [2]:
dataset

,train
0,{'question': 'The Basilica of the Sacred heart...
1,{'question': 'Where is the headquarters of the...
2,{'question': 'What is the primary seminary of ...
3,{'question': 'What is the oldest structure at ...
4,{'question': 'How many BS level degrees are of...
...,...
22996,{'question': 'How many scalar equations were f...
22997,{'question': 'How many vector equations did He...
22998,{'question': 'Who discovered that magnetic and...
22999,{'question': 'At what temperature do weak and ...
